In [ ]:
import Bio
import Bio.Cluster
from Bio import SeqIO
from Bio import AlignIO
from Bio import pairwise2
from scipy.spatial import distance
import os
import threading
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from sklearn.cluster import SpectralClustering, DBSCAN
from collections import Counter
from Bio.Cluster import distancematrix
from Bio.Phylo.TreeConstruction import DistanceCalculator
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
DATA_PATH = "data/DRB_filtered_reads/"

In [ ]:
allFiles = os.listdir(DATA_PATH)

seqs = []

for file in allFiles[3:4]:
    sequences = []

    for record in SeqIO.parse(DATA_PATH + file, "fastq"):
        sequences.append(record.seq)

    print(len(sequences))

    seqs = sequences


In [ ]:
useqs = list(set(seqs))
usegsCounts = [seqs.count(x) for x in useqs]
usegsCounts,len(useqs)

In [ ]:
spectral = SpectralClustering(
        n_clusters=6,
        eigen_solver="arpack",
        assign_labels="cluster_qr",
        affinity="precomputed_nearest_neighbors",
)

dbscan = DBSCAN(eps=.011, metric='precomputed', min_samples=300)

Y  = dbscan.fit_predict(X, sample_weight=usegsCounts)
print(len(X))
print(len(Y))
colors = np.array(["#4B0082","#008080","#2F4F4F","#D8BFD8","#8B4513","#808080","#FF6347","#F0F8FF","#D3D3D3","#4169E1","#DA70D6","#FFFFFF","#DB7093","#87CEEB","#F8F8FF","#FFB6C1","#FAEBD7","#9ACD32","#A9A9A9","red"])

plt.scatter(X[:, 0], X[:, 1], s=1, color=colors[Y])
Y